I will attempt to keep this up to date, but the most up-to-date version will be found on the dropbox.


# Preface

In [ ]:
import numpy as np
import math
import random
import time
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Coordinate Transforms

def cursed_to_spherical(cursed_right, cursed_down):
    """
    Converts the coordinates used in the Compton Cone program, which are in degrees (down from up, right from
        forward) to spherical coordinates, which are in radians and start straight up and then do (to the left
        from forward, depression from up).
    """
    depression = cursed_down * (2 * np.pi / 360)
    azimuth = cursed_right * -1 * (2 * np.pi / 360)
    return azimuth, depression

def spherical_to_cartesian(azimuth, depression):
    dx = np.sin(azimuth) * np.sin(depression)
    dy = np.cos(azimuth) * np.sin(depression)
    dz = np.cos(depression)
    return dx, dy, dz

In [ ]:
# Data Cleaning and Consolidation

def clean_position_data(positions_file):
    """
        The position data contains an awful lot of repeats. This should clean up most of them.
        It returns the (X, Y, Z, Azimuth to the left of center) for the robot as well as
            the time that each position was at (with the start of the program being t=0).
    """

    positions = pd.DataFrame({'Time' : [], 'X' : [], 'Y' : [], 'Z' : [], 'Facing' : []})

    need_start_time = True #We still need to get the start time
    start_time = -1
    for i in range(0,len(positions_file)):
        if positions_file['Frame ID'][i] == "odom_combined":
            #print(i, positions_file['Frame ID'][i])
            if need_start_time:
                start_time = positions_file['Time'][i]
                need_start_time = False

            X = positions_file['X'][i]
            Y = positions_file['Y'][i]
            Z = positions_file['Z'][i]
            F = positions_file['Facing'][i]
            t = positions_file['Time'][i] - start_time

            new_position = pd.DataFrame({'Time' : [t], 'X' : [X], 'Y' : [Y], 'Z' : [Z], 'Facing' : [F]})

            if (np.abs(X) > 0.005 or np.abs(Y) > 0.005):
                positions = pd.concat([positions, new_position], ignore_index = True)
            elif t < 300:
                positions = pd.concat([positions, new_position], ignore_index = True)

    return positions

def get_rays(lines_data, simple_positions_data):
    """
        Goes through each reported line and determines what time it occurred at. It then finds a position point
            from 1 second before and uses that as the position.
    """

    full_clean_data = []

    for i in range(0,len(lines_data['Time'])):
        line_time = lines_data['Time'][i]
        line_azimuth, line_depression = cursed_to_spherical(lines_data['Azimuth'][i], lines_data['Depression'][i])
        print(i, lines_data['Azimuth'][i], lines_data['Depression'][i])
        looking_for_time = True #We are looking for the position data with a matching time

        for j in range(0, len(simple_positions_data['Time'])):
            if looking_for_time:
                if np.abs(simple_positions_data['Time'][j] + 1 - line_time) <= 1:
                    ray = (0, 0, 0, line_azimuth, line_depression) #Initial guess
                    looking_for_time = False #We found an initial time that works
                    x = simple_positions_data['X'][j]
                    y = simple_positions_data['Y'][j]
                    z = simple_positions_data['Z'][j]
                    base_azimuth = simple_positions_data['Facing'][j]
                    ray = (x, y, z, line_azimuth + base_azimuth, line_depression)
                    full_clean_data.append(ray)
                    #print(i, "     ", ray)

    return full_clean_data

In [ ]:
# Location Finding

def single_ray_distance(point, ray):
    base_to_point = (ray[0]-point[0], ray[1]-point[1], ray[2]-point[2]) #v-p
    unit_vector_direction = (spherical_to_cartesian(ray[3], ray[4]))
    cross_product = np.cross(base_to_point, unit_vector_direction)
    distance = math.sqrt(sum(pow(element, 2) for element in cross_product))
    return distance

def total_distance(point, lines):
    total_distance = 0
    for line in lines:
        total_distance += single_ray_distance(point, line)
    return total_distance

def find_closest_point(rays):
  """
    This will probably end up needing information about where to start guessing, but that is a problem for future me.
  """
    estimated_location = minimize(total_distance, (0, 0, 0.5), args=rays).x
    return estimated_location

# Inputs

In [ ]:
lines = pd.read_csv('example_directions.csv')
  #This is the directions to the source with 'Time', 'Azimuth', and 'Depression'
elaborate_positions = pd.read_csv('example_positions.csv')
  #This is the positions with a lot of excess data. It will be of the form [Time, Frame ID, X, Y, Z, Azimuth]

simple_positions = clean_position_data(elaborate_positions)
    #This has cleaned out all of the repeated positions, hopefully
true_data = get_rays(lines, simple_positions)
    #This should have all of the data: the position of the robot and the absolute direction of the source from that position


print(find_closest_point(true_data))